# DNN with Hyper-Parameters Tuning

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

#hyper-parameter tuning
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
LOG_DIR = f"{int(time.time())}" # folder names as a timestamp

UFC_FINAL = pd.read_csv("UFC_FINAL.csv")

# encode blue and red as 1 and 0
UFC_FINAL["Winner"] = UFC_FINAL["Winner"].replace("Blue", 1)
UFC_FINAL["Winner"] = UFC_FINAL["Winner"].replace("Red", 0)

# train/test split
X = UFC_FINAL.drop("Winner", axis=1).values
y = UFC_FINAL["Winner"].values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=101)

# scaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape} | X_test shape: {X_test.shape} | y_train shape: {y_train.shape} | y_test shape: {y_test.shape}")

# function to build the model (argument: hyper-parameter)
def build_model(hp):
    # model
    model = Sequential()
    
    # first layer's no. of neurons = hp.Int range of values to test
    model.add(Dense(hp.Int("input_units", min_value=30,max_value=256,step=16), activation='relu'))
    
    # range of 1 to 10 layers to test
    for i in range(hp.Int("no. Of Hidden Layers", 1, 10)):
        # for each layer, again test range of neurons
        model.add(Dense(hp.Int(f"Hidden_layer_{i+1}_units", min_value=30,max_value=256,step=16), activation='relu'))

    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

tuner = RandomSearch(
        build_model, # name of the function that builds the model
        objective="val_acc", # the thing that we're interested to trace
        max_trials = 2, # no. of combinations to try
        executions_per_trial = 1, # no. of times to train each combination (true avg)
        directory=LOG_DIR) # directory to save the outputs

from tensorflow.keras.callbacks import EarlyStopping # prevent diverge of loss & val_loss
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=16)

X_train shape: (3476, 42) | X_test shape: (1159, 42) | y_train shape: (3476,) | y_test shape: (1159,)


In [4]:
tuner.search(x=X_train,
             y=y_train,
             epochs = 100,
#             batch_size = 64,
             callbacks=[early_stop],
             validation_data=(X_test,y_test))

Train on 3476 samples, validate on 1159 samples
Epoch 1/500
3476/3476 [==============================] - ETA: 1:41 - loss: 0.6977 - acc: 0.312 - ETA: 7s - loss: 0.6855 - acc: 0.6298  - ETA: 3s - loss: 0.6653 - acc: 0.676 - ETA: 2s - loss: 0.6512 - acc: 0.681 - ETA: 1s - loss: 0.6505 - acc: 0.676 - ETA: 0s - loss: 0.6480 - acc: 0.673 - ETA: 0s - loss: 0.6441 - acc: 0.677 - ETA: 0s - loss: 0.6398 - acc: 0.679 - ETA: 0s - loss: 0.6374 - acc: 0.680 - ETA: 0s - loss: 0.6379 - acc: 0.677 - 2s 583us/sample - loss: 0.6382 - acc: 0.6761 - val_loss: 0.6315 - val_acc: 0.6644
Epoch 2/500
3476/3476 [==============================] - ETA: 1s - loss: 0.5584 - acc: 0.781 - ETA: 0s - loss: 0.6098 - acc: 0.691 - ETA: 0s - loss: 0.6121 - acc: 0.689 - ETA: 0s - loss: 0.6121 - acc: 0.687 - ETA: 0s - loss: 0.6071 - acc: 0.692 - ETA: 0s - loss: 0.6115 - acc: 0.685 - ETA: 0s - loss: 0.6057 - acc: 0.689 - ETA: 0s - loss: 0.6089 - acc: 0.686 - ETA: 0s - loss: 0.6135 - acc: 0.681 - ETA: 0s - loss: 0.6142 - acc: 

Train on 3476 samples, validate on 1159 samples
Epoch 1/500
3476/3476 [==============================] - ETA: 1:57 - loss: 0.6953 - acc: 0.406 - ETA: 19s - loss: 0.6578 - acc: 0.625 - ETA: 8s - loss: 0.6437 - acc: 0.6562 - ETA: 5s - loss: 0.6324 - acc: 0.672 - ETA: 3s - loss: 0.6381 - acc: 0.667 - ETA: 2s - loss: 0.6325 - acc: 0.674 - ETA: 2s - loss: 0.6305 - acc: 0.677 - ETA: 2s - loss: 0.6327 - acc: 0.675 - ETA: 2s - loss: 0.6303 - acc: 0.676 - ETA: 1s - loss: 0.6226 - acc: 0.683 - ETA: 1s - loss: 0.6216 - acc: 0.683 - ETA: 1s - loss: 0.6233 - acc: 0.681 - ETA: 0s - loss: 0.6227 - acc: 0.682 - ETA: 0s - loss: 0.6217 - acc: 0.682 - ETA: 0s - loss: 0.6242 - acc: 0.682 - ETA: 0s - loss: 0.6264 - acc: 0.678 - ETA: 0s - loss: 0.6251 - acc: 0.680 - ETA: 0s - loss: 0.6252 - acc: 0.679 - ETA: 0s - loss: 0.6217 - acc: 0.681 - ETA: 0s - loss: 0.6220 - acc: 0.680 - 3s 756us/sample - loss: 0.6225 - acc: 0.6804 - val_loss: 0.6153 - val_acc: 0.6644
Epoch 2/500
3476/3476 [==========================

INFO:tensorflow:Oracle triggered exit


In [5]:
print(tuner.results_summary()) # top 10 trials
print(tuner.get_best_hyperparameters()[0].values) #values of best hyper-parameters
#print(tuner.get_best_models()[0].summary())

None
{'input_units': 46, 'no. Of Hidden Layers': 6, 'Hidden_layer_1_units': 30, 'Hidden_layer_2_units': 30, 'Hidden_layer_3_units': 30, 'Hidden_layer_4_units': 30, 'Hidden_layer_5_units': 30, 'Hidden_layer_6_units': 30}
